<a href="https://colab.research.google.com/github/akshatshaw/LLaMA/blob/main/model_pretrain_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/akshatshaw/LLaMA

Cloning into 'LLaMA'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 43 (delta 17), reused 34 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 1.67 MiB | 9.47 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [3]:
cd LLaMA/

/content/LLaMA


In [4]:
import torch
from model import *
from utils import *
# args.device = "cpu"
# args.vocab_size = 150000

In [6]:
model = Model(args).to(args.device)

In [ ]:
# !pip install transformers
# !pip install tiktoken

In [5]:
from transformers import AutoTokenizer, PreTrainedTokenizerFast

tokenizer = AutoTokenizer.from_pretrained("akshatshaw/LLaMA_hin1")
# import tiktoken
# tokenizer = tiktoken.get_encoding("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/13.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

In [7]:
start_context = "आपके द्वारा चुनी गई भाषा में लिखना आसान बनाता है|"
text_to_token_ids(start_context, tokenizer)

tensor([[ 6591,  6410, 17978,  6295,  7096,  6078, 12466,  8058, 10779,  6073,
            56]], device='cuda:0')

In [8]:
torch.manual_seed(10)
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=args.max_seq_len
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 आपके द्वारा चुनी गई भाषा में लिखना आसान बनाता है | लास्‍लो परेशानी फीयर रोकने इलाह तोहरे एकला गरीब कॉरेसपोंडेंस सौदर्य


In [9]:
file_path = "Full_text_Bible.txt"
with open(file_path, "r", encoding="utf-8") as file:
 text_data = file.read()

In [10]:
total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))
print("Characters:", total_characters)
print("Tokens:", total_tokens)

Characters: 3693013
Tokens: 936185


In [11]:
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

In [12]:
dataset = ModelDataset(text_data, tokenizer, args.max_seq_len, args.max_seq_len)
dataset.__len__()

3656

In [13]:
torch.manual_seed(123)
train_loader = create_dataloader_v1(
 train_data,
 tokenizer,
 batch_size=2,
 max_length=args.max_seq_len,
 stride=args.max_seq_len,
 drop_last=True,
 shuffle=True,
 num_workers=0
)
val_loader = create_dataloader_v1(
 val_data,
 tokenizer,
 batch_size=2,
 max_length=args.max_seq_len,
 stride=args.max_seq_len,
 drop_last=False,
 shuffle=False,
 num_workers=0
)

In [14]:
print("Train loader:")
for x, y in train_loader:
 print(x.shape, y.shape)
 break
print("\nValidation loader:")
for x, y in val_loader:
 print(x.shape, y.shape)
 break

Train loader:
torch.Size([2, 256]) torch.Size([2, 256])

Validation loader:
torch.Size([2, 256]) torch.Size([2, 256])


In [15]:
def calc_loss_batch(input_batch, target_batch, model, device= args.device):
 input_batch = input_batch.to(device)
 target_batch = target_batch.to(device)
 logits = model(input_batch)
 loss = torch.nn.functional.cross_entropy(
    logits.flatten(0, 1), target_batch.flatten()
 )
 return loss

In [16]:
def calc_loss_loader(data_loader, model, device= args.device, num_batches=None):
    total_loss = 0
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(
                input_batch, target_batch, model, device
            )
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches # Averages the loss over all batches

In [17]:
torch.manual_seed(10)
with torch.no_grad():
  train_loss = calc_loss_loader(train_loader, model)
  val_loss = calc_loss_loader(val_loader, model)
print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Training loss: 12.073393342854978
Validation loss: 12.057029878626988


In [18]:
def train_model_simple(model, train_loader, val_loader,optimizer, device, num_epochs,eval_freq, eval_iter, start_context, tokenizer):
    train_losses, val_losses, track_tokens_seen = [], [], []   # tracking losses and token seen
    tokens_seen, global_step = 0, -1
    for epoch in range(num_epochs): # Main training loop
        model.train()
        for input_batch, target_batch in train_loader: # this loop iterates in batches

            # model.apply(lambda module: module.reset_kv_cache() if isinstance(module, MHA) else None)

            optimizer.zero_grad()           # reset loss gradient from previous batch iteration
            loss = calc_loss_batch(
                 input_batch, target_batch, model, device
            )
            loss.backward() # loss gradient
            optimizer.step() # update model weights using loss gradients
            tokens_seen += input_batch.numel()
            global_step += 1
            if global_step % eval_freq == 0: # Optional evaluation
                train_loss, val_loss = evaluate_model(model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                f"Train loss {train_loss:.3f}, "
                f"Val loss {val_loss:.3f}")
        generate_and_print_sample(model, tokenizer, device, start_context)
    return train_losses, val_losses, track_tokens_seen

In [19]:
# Evaluating the validation losses
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()     # dropouts are disabled
    with torch.no_grad(): # disablibng the graddient tracking
        train_loss = calc_loss_loader(
                    train_loader, model, device, num_batches=eval_iter
            )
        val_loss = calc_loss_loader(
                     val_loader, model, device, num_batches=eval_iter
            )
    model.train()
    return train_loss, val_loss

In [20]:
# Generating and printing a sample text using the genearte_and_print_sample function
def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = args.max_seq_len
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
            )
    decoded_text = token_ids_to_text(token_ids, tokenizer)
    print(decoded_text.replace("\n", " "))
    model.train()

In [22]:
# torch.cuda.empty_cache()

In [23]:
# Ready.... Set.... Go!
torch.manual_seed(123)

# model = GPTModel(GPT_CONFIG_124M)
# model.to(device)

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=0.0004, weight_decay=0.1
)
num_epochs = 5

train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, args.device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="एक समय की बात है", tokenizer=tokenizer
)

Ep 1 (Step 000000): Train loss 11.480, Val loss 11.415
Ep 1 (Step 000005): Train loss 9.013, Val loss 8.839
Ep 1 (Step 000010): Train loss 7.528, Val loss 7.202
Ep 1 (Step 000015): Train loss 6.782, Val loss 6.649
Ep 1 (Step 000020): Train loss 6.832, Val loss 6.443
Ep 1 (Step 000025): Train loss 6.585, Val loss 6.337
Ep 1 (Step 000030): Train loss 6.266, Val loss 6.254
Ep 1 (Step 000035): Train loss 6.140, Val loss 6.257
Ep 1 (Step 000040): Train loss 6.274, Val loss 6.131
Ep 1 (Step 000045): Train loss 6.026, Val loss 6.036
Ep 1 (Step 000050): Train loss 6.063, Val loss 6.018
Ep 1 (Step 000055): Train loss 5.901, Val loss 5.959
Ep 1 (Step 000060): Train loss 5.940, Val loss 5.915
Ep 1 (Step 000065): Train loss 5.667, Val loss 5.894
Ep 1 (Step 000070): Train loss 5.565, Val loss 5.779
Ep 1 (Step 000075): Train loss 5.551, Val loss 5.584
Ep 1 (Step 000080): Train loss 5.414, Val loss 5.527
Ep 1 (Step 000085): Train loss 5.454, Val loss 5.473
Ep 1 (Step 000090): Train loss 5.490, Val lo

KeyboardInterrupt: 

In [28]:
generate_and_print_sample(model, tokenizer, args.device, "एक समय की बात |")

एक समय की बात | चांदी चांदी एक एक एक एक एक चांदी चांदी चांदी पांच पांच पांच पांच पांच चांदी चांदी चांदी पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच पांच


In [30]:
def generate(model, idx, max_new_tokens, context_size,
    temperature=0.0, top_k=None, eos_id=None):
    for _ in range(max_new_tokens): # The for loop is the same as before: gets logits and only focuses on the last time step.
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
            logits = logits[:, -1, :]
            if top_k is not None: # filters logits to only consider the top-k tokens
                top_logits, _ = torch.topk(logits, top_k)
                min_val = top_logits[:, -1]
                logits = torch.where(
                    logits < min_val,
                    torch.tensor(float('-inf')).to(logits.device),
                    logits
                )
            if temperature > 0.0: # applies temperature scaling
                logits = logits / temperature
                probs = torch.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)
            else:
                idx_next = torch.argmax(logits, dim=-1, keepdim=True)
            if idx_next == eos_id: # Stops generating early if end-of-sequence token is encountered
                break
            idx = torch.cat((idx, idx_next), dim=1)
    return idx

In [36]:
token_ids_out = generate(
    model=model,
    idx=text_to_token_ids("एक समय की बात ", tokenizer),
    max_new_tokens=25,
    context_size=args.max_seq_len,
    top_k=15,
    temperature=4
)
print("Output text:\n", token_ids_to_text(token_ids_out, tokenizer))

Output text:
 एक समय की बात कों जाली हि झटपट आदि बनाता डालेंगे डालेंगे देवताओं खा सकेंगें दसवें दसवें पशुओं बनाने लगे वैसा बड़ी बड़ी पहिली दक्खिन मैदे ड़ों ईश्वरों बनी
